In [1]:
import pandas as pd
import geopandas as gpd
import contextily as cx
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from shapely.geometry import Point, LineString, MultiLineString
geojson = 'GeoJSON'

In [2]:
mesh = gpd.read_file('../data/2019/mesh500/HDDSWH5339/MESH05339.shp', crs=4612).to_crs(2451)

In [5]:
mesh.geometry[0]

NotImplementedError: 

In [52]:
kokusei = pd.DataFrame()
for x in mesh.MESH1_ID.unique():
    kokusei = pd.concat([kokusei,
        pd.read_csv(f"../data/2019/kokutyou500/tblT000847H{x}.txt", encoding='sjis', skiprows=[1])])
kokusei = kokusei[['KEY_CODE', 'T000847001']]
kokusei.columns = ['KEY_CODE', 'people']
kokusei.KEY_CODE = kokusei.KEY_CODE.astype(str)
kokusei = pd.merge(mesh[['KEY_CODE', 'geometry']], kokusei, on='KEY_CODE')
kokusei['area_t'] = kokusei.area

In [53]:
keizai = pd.DataFrame()
for x in mesh.MESH1_ID.unique():
    keizai = pd.concat([keizai,
        pd.read_csv(f"../data/2019/keizai500/tblT000389H{x}.txt", encoding='sjis', skiprows=[1])])
keizai.columns = ['KEY_CODE', 'company', 'people']
keizai.KEY_CODE = keizai.KEY_CODE.astype(str)
keizai = pd.merge(mesh, keizai, on='KEY_CODE')
keizai['area_t'] = keizai.area

In [54]:
stations = gpd.read_file('../data/railway/ekidata.geojson', crs=4326).to_crs(2451)

In [55]:
tokyoeki = stations[stations.station_name.str.contains('東京')].geometry.values[0]

In [56]:
stations[stations.station_name.str.contains('あざみ野')]

,company_cd,company_name,line_cd,line_name,line_name_h,station_cd,station_g_cd,station_name,e_sort,geometry
5664,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600316,2600316,あざみ野,2600316,POINT (-25368.430 -47820.172)
7908,130,横浜市交通局,99316,ブルーライン,横浜市高速鉄道1号線・3号線,9931601,2600316,あざみ野,9931601,POINT (-25347.580 -47883.357)


In [57]:
x = stations[stations.line_cd.isin([26003])].copy()
x.geometry = stations.geometry.buffer(1000)
x = gpd.overlay(kokusei, x)
x['ans'] = x.people * (x.area / x.area_t)
x = x.groupby('station_cd').sum().reset_index()[['station_cd', 'ans']]
po = pd.merge(stations, x, on='station_cd')
po = po.drop_duplicates(subset='station_g_cd')
po = po.sort_values(['station_cd', 'e_sort'])
po = po.iloc[1:, :]
po.ans = round(po.ans).astype(int)
po['tokyo'] = round(po.geometry.distance(tokyoeki), -3).astype(int)

In [58]:
syugyo_buffer = pd.read_csv('output/syugyo-buffer.csv')

In [59]:
syugyo_buffer.columns = ['tokyo', 'syugyo'] 
x = syugyo_buffer.copy()
x.syugyo = x.syugyo - 30
po2 = pd.merge(po, x, on='tokyo', how='left')

In [60]:
po2.syugyo = po2.syugyo.fillna(70)
po2['genjitsu'] = po2.ans * 0.5 * (po2.syugyo / 100)

In [61]:
(po2.genjitsu.sum() * 0.3 / 28 ) / 1526

1.628269202864632

In [62]:
x = stations[stations.line_cd.isin([26003])].copy()
x.geometry = stations.geometry.buffer(2000)
x = gpd.overlay(kokusei, x)
x['p2000'] = x.people * (x.area / x.area_t)
x = x.groupby('station_cd').sum().reset_index()[['station_cd', 'p2000']]
po3 = pd.merge(po2, x, on='station_cd')

In [63]:
csv1 = 'output/jyousya/denen.csv'
#po3[['station_cd', 'station_name']].to_csv(csv1, index=None)

In [64]:
x = pd.read_csv(csv1).drop('station_name', 1)
x = pd.merge(po3, x, on='station_cd').iloc[:, 2:].drop(['line_name_h', 'station_g_cd', 'e_sort'], 1)
x.p2000 = x.p2000.astype(int)
x['g2'] = x.genjitsu.mask(x.f1 == 1, (x.p2000 - x.ans) * 0.5 * ((x.syugyo - 10) / 100)).astype(int)
print('混雑率 :', (x.g2.sum() * 0.3 / 28 ) / 1526)

混雑率 : 2.0796386444486052


In [65]:
po3

,company_cd,company_name,line_cd,line_name,line_name_h,station_cd,station_g_cd,station_name,e_sort,geometry,ans,tokyo,syugyo,genjitsu,p2000
0,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600302,2600302,池尻大橋,2600302,POINT (-13493.353 -38753.508),63533,8000,70,22236.550,225993.783060
1,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600303,2600303,三軒茶屋,2600303,POINT (-14777.091 -39515.526),72582,10000,70,25403.700,249370.997856
2,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600304,2600304,駒沢大学,2600304,POINT (-15544.657 -40650.817),58361,11000,65,18967.325,231077.023932
3,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600305,2600305,桜新町,2600305,POINT (-17077.760 -40848.259),55590,12000,60,16677.000,204829.971091
4,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600306,2600306,用賀,2600306,POINT (-18061.735 -41426.544),48011,13000,56,13443.080,176838.171829
5,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600307,2600307,二子玉川,2600307,POINT (-18721.206 -43050.255),37340,15000,49,9148.300,162503.382587
6,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600308,2600308,二子新地,2600308,POINT (-19116.161 -43581.270),45409,15000,49,11125.205,169137.845853
7,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600309,2600309,高津,2600309,POINT (-19606.678 -43995.791),55022,16000,46,12655.060,175672.980985
8,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600310,1130310,溝の口,2600310,POINT (-20191.660 -44372.574),55037,17000,44,12108.140,178861.687081
9,16,東急電鉄,26003,東急田園都市線,東急田園都市線,2600311,2600311,梶が谷,2600311,POINT (-20610.145 -45023.619),51170,17000,44,11257.400,190648.170582
